In [62]:
import pandas as pd
import nltk
nltk.download('punkt')
import numpy as np


[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [86]:
df = pd.read_csv("fr_crous_logement_france_entiere.csv", sep =';')
df

,id,title,short_desc,zone,infos,services,contact,address,mail,phone,openinghours,interneturl,appointmenturl,virtualvisiturl,bookingurl,troubleshootingurl,geocalisation,photo,regions
0,205,Résidence Vauban,Guyancourt,YVELINES,Description La résidence est située au cœur de...,Laverie \r\n Salles de travail \r\n Salles d...,Résidence Vauban\r\n 1 boulevard d’Alembert782...,1 boulevard d’Alembert 78280 Guyancourt,residence.vauban@crous-versailles.fr,01 80 73 04 00,NaN,https://www.crous-versailles.fr/,https://www.crous-versailles.fr/,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.7796516418,2.0441513062",http://ephoto.nuonet.fr/link/Dz4FNVwrUWMAYAE5A...,Île-de-France
1,207,Résidence Le Square de l'Echiquier,Cergy-Pontoise,VAL D'OISE,DescriptionRésidence universitaire nouvellemen...,Wifi \r\n Laverie \r\n salle de sport \r\n ...,Résidence Le Square de l'échiquier\n9 square d...,9 square de l'échiquier 95800 Cergy-Pontoise,residence.lacroix@crous-versailles.fr,01 30 30 38 53,NaN,https://www.crous-versailles.fr/,https://www.crous-versailles.fr/,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"49.0482978821,2.02688694",http://stockcrous.fr/static-images/10bcd8bb8ec...,Île-de-France
2,214,Résidence Les Bas Longchamps,Bagneux,HAUTS DE SEINE,Description Résidence universitaire située à p...,Laverie \r\n Salles de travail et de détent...,Résidence de Bagneux /Les Bas de Longchamps\r\...,2 rue Abraham Lincoln 92220 Bagneux,residence.bagneux@crous-versailles.fr,01 86 63 00 90,NaN,https://www.crous-versailles.fr/,https://www.crous-versailles.fr/,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.7906913757,2.3169708252",https://admin-v2.crous-mobile.fr//media/files/...,Île-de-France
3,225,Résidence de l’Ile,Bures-sur-Yvette,ESSONNE,Description Résidence universitaire située à p...,"Salles de détente, une avec télévision \r\n ...",Résidence de l’Ile\n14 rue du Docteur Collé - ...,14 rue du Docteur Collé - bâtiment 227 du camp...,residence.ile@crous-versailles.fr,01 69 07 68 96,NaN,https://www.crous-versailles.fr/,https://www.crous-versailles.fr/,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.698802948,2.1578958035",https://admin-v2.crous-mobile.fr//media/files/...,Île-de-France
4,228,Résidence La Pacaterie,Orsay,ESSONNE,Description Résidence universitaire longée par...,"Salles de détente (télévision, ate...",Résidence La Pacaterie\nBâtiment 499 - rue de ...,Bâtiment 499 - rue de la Pacaterie 91400 Orsay,residence.pacaterie@crous-versailles.fr,01 69 15 75 98,NaN,https://www.crous-versailles.fr/,https://www.crous-versailles.fr/,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.7015419006,2.1846776009",https://admin-v2.crous-mobile.fr//media/files/...,Île-de-France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,349,Cité Grémillon (secteur d'Hérouville-Saint-Clair),NaN,Herouville Saint Clair,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,Gratuits (Sauf € = Payants),Cité Grémillon (Secteur d'Hérouville-Saint-Cla...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,herouville@crous-normandie.fr,02 31 47 61 23,NaN,https://www.crous-normandie.fr/,NaN,NaN,NaN,NaN,"49.2032051086,-0.3392136097",https://admin-v2.crous-mobile.fr//media/crous-...,Normandie
784,331,Résidence Antide Janvier,"Centre ville, sur les quais",Besançon - Centre ville,"Localisation :\r\n Au centre ville, à proximit...",Wifi Inclus dans le loyerPoint restoParking,Résidence Antide Janvier \r\n 49 T1 et de 27 T...,6 rue Antide Janvier 25000 Besançon,pole-hebergement-besancon@crous-besancon.fr,03 81 48 47 50,NaN,https://www.crous-bfc.fr/,NaN,http://www.crous-bfc.fr/logements/logement-cro...,NaN,https://www.crous-bfc.fr/,"47.2365074158,6.0153431892",https://admin-v2.crous-mobile.fr//media/files/...,Bourgogne-Franche-Comté
785,335,Résidence Jean 

In [ ]:
#On réapplique la même logique pour extraire les surfaces.

In [87]:
List_check = ["m²"]

In [88]:
print(df['infos'][200])

74 logements étudiants 5 T4 en colocation 5 Logements PMR pour personne à mobilité réduite 
 
Logement T2 :


Logement T3 :


In [89]:

def string_checking(string, check):
    result = 0 
    if string.find(check) == -1:
        result = 0
    else: 
        result = 1
    return result

In [90]:
def word_ban(string, liste):
    result = 0 
    for check in liste: 
        if string_checking(string, check) == 1: 
            result = 1 
        else: 
            result = 0
    return result 
            

In [93]:
def surface (word):
    Surface = [] #On crée une liste qui stockera la surface
    #try:
    for i in range(0,len(word)): #on regarde dans chaque block de caractère 
        index = word[i].find("m²") #on cherche l'index du m² normalement pour toutes les surfaces il y est 
        search = 0 #il nous servira à arrêter la boucle while 
        if word[i].find("m²") != -1: 
            while search == 0: #Tant que notre variable de contrôle est égale à 0, on va chercher la surface (ie : tant que ce sont des chiffres, on va chercher la nature (chffre ou non) de ce qui se situe à gauche.
                search_index = word[i][index - 1] #On cherche l'index de ce qui se trouve à gauche du m²
                try: #Try permet de tester si c'est un chiffre tout en ne bloquant pas les opérations avec une erreur
                    search_index = int(search_index) #On regarde si à gauche de l'ancien index c'est un entier ou non (Dans le cas contraire on sera dans except).
                    index = index - 1 #on remonte de 1 l'index pour voir si ce qui se trouve à gauche de notre chiffre est également un chiffre.
                except: #si ce n'est pas un entier à gauche de l'euros ou du dernier chiffre identifié. 
                    if word[i][index - 1] == "." :
                        index = index - 2 #On va regarde ce qui est avant le point pour trouver le reste du montant
                        search_index = word[i][index - 1] #Sysiphe remonte sa pierre et recommence le même processus qu'avant
                        try: #on regarde de nouveau si à gauche du point c'est un entier
                            search_index = int(search_index) 
                            index = index - 1 
                        except:
                            search = 1 #si ce n'est pas un chiffre ie une lettre on arrête
                            index = -1 
    
                    else: #si ce n'est pas un point on arrête le compte. 
                        search = 1
                #print(word[i][index : word[i].find("euros")])

            if word[i][index : word[i].find("m²")]!= '': #on regarde le fait que le token ne soit pas juste un '€'
                Surface.append(float(word[i][index : word[i].find("m²")])) #On ajoute à la surface la surface trouvée
    #except:
        #if word[i] != "€" :
            #List_banned_word.append(word[i])
            #print(List_banned_word)
            #rent(word)
    return Surface

In [94]:
def surface_find(residence): 
    residence = residence.replace(' ', '')
    residence = residence.replace('(', '')
    residence = residence.replace(')', '')
    residence = residence.replace('m²à', 'm² ')
    residence = residence.replace('m²', 'm² ')
    residence = residence.replace('m2', 'm²') #unificier les notations de m2
    residence = residence.replace('€€', '€')
    residence = residence.replace('€', '€ ')
    residence = residence.replace(',', '.')
    residence = residence.replace(':', '')
    word =  nltk.word_tokenize(residence, language='french') #on transforme notre scripts en block de mots. ceux qui nous intéressent ont un euros à la fin (mais il y en a qui ont un euros à la fin et qui ne nous intéresse pas
    return surface (word)

In [95]:
#On fait qq tests sur des lignes tirées au hasard pour vérifier les résultats
print(surface_find(df["infos"][58]))
print(surface_find(df["infos"][90]))
print(surface_find(df["infos"][349]))

[23.0, 23.0]
[]
[19.0, 27.0]


In [96]:
print(df["infos"][58]) #OK on trouve les bonnes surfaces 
print(df["infos"][90]) #OK on trouve les bonnes surfaces 
print(df["infos"][349]) #OK on trouve les bonnes surfaces 

Située dans le coeur historique de Clermont-Ferrand, ces 9 logements sont répartis en 5 T1 (pour une personne seule) et 4 T1bis (pour deux personnes). 
STUDIOS T1 et T1 bis
SUPERFICIE : 23 m² (à partir de 23 m² pour un T1 bis) 
 LOYER MENSUEL : T1– 305 € / T1bis – 346 € (l’électricité pour l’usage courant et le chauffage restent à la charge de l’étudiant) 
 DÉPÔT DE GARANTIE : T1-310 € / T1bis – 350 € 
 DURÉE DU CONTRAT : 12 mois du (1er septembre au 31 août) 
 ÉQUIPEMENT : Kitchenette (plaques chauffantes, réfrigérateur, évier), sanitaire individuel (douche, WC, Lavabo), prise téléphone, prise télévision, accès Internet 
 Lieux d’études proches : École Supérieure de Commerce 
Transports : Bus 12, 27, 35 et 36 arrêt Montlosier, Tram A arrêt Montlosier 
 Ces logements ouvrent tous droit à l'Allocation Personnalisée au Logement (APL). Tous les renseigments sur les aides au logement sur www.caf.fr.
Localisation
 18 bd de la Borie 87100 LIMOGES Services administratifs Résidence de La Borie

In [97]:
a = 0
b = 0
Surface = []
for j in range(0,df.shape[0]):
    try:
        Surface.append(surface_find(df["infos"][j]))
        b = b + 1
    except: 
        Surface.append("NAN")
        a = a + 1
df['Surface'] = Surface
print("échec pour ",a)
print("réussite pour ", b)
print("total ", a + b) 

échec pour  11
réussite pour  777
total  788


In [98]:
#Je ne comprend pas pourquoi on trouve les même valeurs que pour le loyer ? 

In [99]:
df['Surface'] # la colonne affiche des valeurs qui sont cohérentes pour des surfaces

0                                       []
1                                       []
2                                   [28.0]
3                                   [18.0]
4                                       []
                      ...                 
783    [9.0, 18.0, 30.0, 49.0, 65.0, 69.0]
784                           [19.0, 35.3]
785                     [18.0, 36.0, 51.5]
786                                    NAN
787                                     []
Name: Surface, Length: 788, dtype: object

In [111]:
print(df["infos"][783]) #ok on a toutes les surfaces 
print(df["infos"][785])#ok il existe bien une surface de 51,5 m2 

BP 54 - Avenue de la Grande Cavée 14202 Hérouville-Saint-Clair cedex
 À proximité des centres commerciaux (dont 2 centres de proximité et un centre régional), de la Caf et d’une offre complète de services culturels (bibliothèque, cinéma avec accès privilégié pour les résidents du site d’Hérouville -Se renseigner auprès du Café des images-, salle de concert, théâtre). 
Accès
 Tram "T1" - Arrêt château d’eau et bus - lignes 4 et 9 
   



Surface moyenne
Loyer


Chambre simple de 9m²
249.85 €


T1 de 18m²
312.90 €


T1 BIS de 30m²
388.60 €


T2 de 49m²
467.40 €


T3 de 65m²
525.00 €


T4 de 69m²
556.14 €



 Les T3 et T4 sont reservés en priorité aux familles avec enfants.
Localisation :
 La résidence Sartre est située au centre ville. Elle est proche de l’IUFM, du Lycée Condorcet, de l’ESTA, de l’ISTC et de l’IFSI. 
Description :
 La résidence Sartre propose 129 T1, 11 T1 bis et 2 T2 et dispose d’aménagement pour handicapés. 
Moyens d'accès :
 Bus : 1 - S5 - 7 - 31 
Types de logement :


In [100]:
import math

a = df['Surface'][120]


print(a)
print(len(a))

[]
0


In [107]:
def remove_fun(residence):
    lenght = residence
    for i in range(0, len(lenght)):
        num = int(residence[i])
        if num <= 7 or num >= 100: # Un surface peut rarement être en dessous de 7 m2 ou au -dessus de 60 (valeurs aberrantes)
            residence.remove(residence[i])
    return residence


In [108]:
for j in range(0,df.shape[0]):
    if df["Surface"][j] != "NAN":
        try : 
            remove_fun(df["Surface"][j])
        except: 
            try: 
                remove_fun(df["Surface"][j])
            except: 
                remove_fun(df["Surface"][j])
                try: 
                    remove_fun(df["Surface"][j])
                except: 
                    remove_fun(df["Surface"][j])

In [109]:
Max = []
Min = []
Mean = []
a = 0
for j in range(0,df.shape[0]):
    if df["Surface"][j] != "NAN" and df["Surface"][j] != []:
        a = a + 1
        Max.append(max(df["Surface"][j]))
        Min.append(min(df["Surface"][j]))
        Mean.append(np.mean(df["Surface"][j]))
    else: 
         Max.append("NaN")
         Min.append("NaN")
         Mean.append("NaN")
df['Max Surface'] = Max
df['Min Surface'] = Min
df['Mean Surface'] =  Mean # on crée les 3 nouvelles colonnes
print(a) # avec des valeurs renseignées dans 474 lignes.

476


In [110]:
df

,id,title,short_desc,zone,infos,services,contact,address,mail,phone,...,virtualvisiturl,bookingurl,troubleshootingurl,geocalisation,photo,regions,Surface,Max Surface,Min Surface,Mean Surface
0,205,Résidence Vauban,Guyancourt,YVELINES,Description La résidence est située au cœur de...,Laverie \r\n Salles de travail \r\n Salles d...,Résidence Vauban\r\n 1 boulevard d’Alembert782...,1 boulevard d’Alembert 78280 Guyancourt,residence.vauban@crous-versailles.fr,01 80 73 04 00,...,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.7796516418,2.0441513062",http://ephoto.nuonet.fr/link/Dz4FNVwrUWMAYAE5A...,Île-de-France,[],NaN,NaN,NaN
1,207,Résidence Le Square de l'Echiquier,Cergy-Pontoise,VAL D'OISE,DescriptionRésidence universitaire nouvellemen...,Wifi \r\n Laverie \r\n salle de sport \r\n ...,Résidence Le Square de l'échiquier\n9 square d...,9 square de l'échiquier 95800 Cergy-Pontoise,residence.lacroix@crous-versailles.fr,01 30 30 38 53,...,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"49.0482978821,2.02688694",http://stockcrous.fr/static-images/10bcd8bb8ec...,Île-de-France,[],NaN,NaN,NaN
2,214,Résidence Les Bas Longchamps,Bagneux,HAUTS DE SEINE,Description Résidence universitaire située à p...,Laverie \r\n Salles de travail et de détent...,Résidence de Bagneux /Les Bas de Longchamps\r\...,2 rue Abraham Lincoln 92220 Bagneux,residence.bagneux@crous-versailles.fr,01 86 63 00 90,...,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.7906913757,2.3169708252",https://admin-v2.crous-mobile.fr//media/files/...,Île-de-France,[28.0],28.0,28.0,28.0
3,225,Résidence de l’Ile,Bures-sur-Yvette,ESSONNE,Description Résidence universitaire située à p...,"Salles de détente, une avec télévision \r\n ...",Résidence de l’Ile\n14 rue du Docteur Collé - ...,14 rue du Docteur Collé - bâtiment 227 du camp...,residence.ile@crous-versailles.fr,01 69 07 68 96,...,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.698802948,2.1578958035",https://admin-v2.crous-mobile.fr//media/files/...,Île-de-France,[18.0],18.0,18.0,18.0
4,228,Résidence La Pacaterie,Orsay,ESSONNE,Description Résidence universitaire longée par...,"Salles de détente (télévision, ate...",Résidence La Pacaterie\nBâtiment 499 - rue de ...,Bâtiment 499 - rue de la Pacaterie 91400 Orsay,residence.pacaterie@crous-versailles.fr,01 69 15 75 98,...,NaN,https://www.crous-versailles.fr/logements/1795-2/,https://www.crous-versailles.fr/,"48.7015419006,2.1846776009",https://admin-v2.crous-mobile.fr//media/files/...,Île-de-France,[],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,349,Cité Grémillon (secteur d'Hérouville-Saint-Clair),NaN,Herouville Saint Clair,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,Gratuits (Sauf € = Payants),Cité Grémillon (Secteur d'Hérouville-Saint-Cla...,BP 54 - Avenue de la Grande Cavée 14202 Hérouv...,herouville@crous-normandie.fr,02 31 47 61 23,...,NaN,NaN,NaN,"49.2032051086,-0.3392136097",https://admin-v2.crous-mobile.fr//media/crous-...,Normandie,"[9.0, 18.0, 30.0, 49.0, 65.0, 69.0]",69.0,9.0,40.0
784,331,Résidence Antide Janvier,"Centre ville, sur les quais",Besançon - Centre ville,"Localisation :\r\n Au centre ville, à proximit...",Wifi Inclus dans le loyerPoint restoParking,Résidence Antide Janvier \r\n 49 T1 et de 27 T...,6 rue Antide Janvier 25000 Besançon,pole-hebergement-besancon@crous-besancon.fr,03 81 48 47 50,...,http://www.crous-bfc.fr/logements/logement-cro...,NaN,https://www.crous-bfc.fr/,"47.2365074158,6.0153431892",https://admin-v2.crous-mobile.fr//media/files/...,Bourgogne-Franche-Comté,"[19.0, 35.3]",35.3,19.0,27.15
785,335,Résidence Jean Paul Sartre,Centre ville,Belfort,Localisation :\r\n La résidence Sartre est sit...,LaverieParking fermé (15€/mois),"Résidence Jean-Paul Sartre \r\n 130 T1, 11 T1 ...",Rue Gaston Deferre 90000 Belfort,pole-he